In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from statsmodels.api import WLS, add_constant

In [5]:
data = pd.read_csv('../data/df-time-agg.csv')
data = data[data["person"]!="ingeborgR"]
data = data[(data["person"] != "adamF") | (data["coin"] != "0.05EUR") | (data["agg"] != 84)]


data_expanded = data.copy()
data_expanded['r'] = data["heads_heads"]+(data['N_start_tails_up']-data["tails_heads"])
data_expanded['m'] = data['N_start_tails_up']+data['N_start_heads_up']
data_expanded['rom'] = data_expanded['r'] / data_expanded['m']
data_expanded['weights'] = (4*data_expanded['m'])

print(len(data_expanded))

3409


In [ ]:
# Define custom scoring function using AIC for WLS
def aic_score_wls(estimator, X, y, weights):
    """
    Custom AIC scoring function for WLS.
    """
    X_const = add_constant(pd.DataFrame(X).astype(float))  # X const
    model = WLS(y, X_const, weights=weights).fit()  # Use WLS
    return -model.aic  # Negate AIC because SequentialFeatureSelector maximizes the score

# Prepare your data
# Assuming 'rom' is the target variable, 'weights' are the weights, and we one-hot encode 'person' and 'coin'
X = pd.get_dummies(data_expanded[['person', 'coin']], drop_first=True)
y = data_expanded['rom']
weights = data_expanded['weights']  # Ensure this column exists in your data

# Initialize the base estimator (LinearRegression for feature selection only)
base_model = LinearRegression()

# Use SequentialFeatureSelector
sfs = SequentialFeatureSelector(
    estimator=base_model,
    n_features_to_select="auto",  # Choose optimal number of features
    direction='backward',          # backward selection
    scoring=lambda est, X, y: aic_score_wls(est, X, y, weights=weights),  # Custom scoring with WLS
    cv=None,                      # No cross-validation for AIC
)

# Fit the feature selector
sfs.fit(X, y)

# Selected features
selected_features = X.columns[sfs.get_support()]

print("Selected features based on WLS AIC:", selected_features)

/Users/rayan/Documents/GitHub/reg-project/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/rayan/Documents/GitHub/reg-project/venv/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 144, in __call__
    score = scorer(estimator, *args, **routed_params.get(name).score)
  File "/var/folders/fd/plcsl8g53jj2g19ljhctv5_m0000gn/T/ipykernel_5885/1162446724.py", line 24, in <lambda>
    scoring=lambda est, X, y: aic_score_wls(est, X, y, weights=weights),  # Custom scoring with WLS
  File "/var/folders/fd/plcsl8g53jj2g19ljhctv5_m0000gn/T/ipykernel_5885/1162446724.py", line 7, in aic_score_wls
    model = WLS(y, X_const, weights=weights).fit()  # Use WLS
  File "/Users/rayan/Documents/GitHub/reg-project/venv/lib/python3.9/site-packages/statsmodels/regression/linear_model.py", line 746, in __